In [1]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.message import Attachment, AttachmentDataJson, AttachmentDataLinks
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID

from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
             didcomm_services = []
#                 DIDCommService(
#                     id = s.id,
#                     service_endpoint = s.service_endpoint,
#                     routing_keys = s.routing_keys,
#                     accept = s.accept
#                 )
#                 for s in did_doc.service
#                 if isinstance(s, DIDCommServicePeerDID)
#            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="#didcommmessaging-0",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Alice get OOB from Mediator

In [5]:
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text

In [6]:
print(oob_url)

https://mediator.rootsid.cloud?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiOWQ5ZjdhMzYtYmYzYS00YWViLWE5MTItMDExZjM3MTc4NzM3IiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNtczU1NVloRnRobjFXVjhjaURCcFptODZoSzl0cDgzV29qSlVteFBHazFoWi5WejZNa21kQmpNeUI0VFM1VWJiUXc1NHN6bTh5dk1NZjFmdEdWMnNRVllBeGFlV2hFLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwibGFiZWwiOiJNZWRpYXRvciIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ


In [7]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
alice_did_for_mediator = await create_peer_did(1,1)
print("Alice's DID:", alice_did_for_mediator)

Alice's DID: did:peer:2.Ez6LSjKTa79FgzbLp8g3U2MmrpQRJpgYTXcgbu5sSKvGK1DMU.Vz6Mkso5Q23mjBA4ayqeFVht6CGZBYQa6e8LXEkmfVsPaKHLq


### Alice request mediate

In [8]:
alice_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)

In [9]:
alice_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_mediate_grant,
    frm = alice_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [10]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

https://mediator.rootsid.cloud


In [11]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, json = json.loads(alice_mediate_grant_packed.packed_msg))


In [12]:
print(resp.json())

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNRMlkwVmlOVTU0WVZsVFpFZDRSa2hGVFRkT1IzUTJSMDV6TVV4TmRUYzVTazFFY2xaMU9XVlRabkJHTGxaNk5rMXJiMHcwUTJkTVl6bDNZMVJ4UVdWbFNuVkRRMjVhVW5reU1scDFPVGhRTkc1dGNVWnhaVFJpTlZoWU1rRXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eU1XeGFSMnhvWkVjNWVVeHVTblppTTFKNllWZFJkVmt5ZUhaa1YxRnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGZ3dJelpNVTNRMlkwVmlOVTU0WVZsVFpFZDRSa2hGVFRkT1IzUTJSMDV6TVV4TmRUYzVTazFFY2xaMU9XVlRabkJHIiwiYXB2IjoiZ1VrT2lJUTVUbmxLUWVDcE1xYVFKbGlRdmpWWTBHUXcyNlhRa01ZblAtYyIsInNraWQiOiJkaWQ6cGVlcjoyLkV6NkxTdDZjRWI1TnhhWVNkR3hGSEVNN05HdDZHTnMxTE11NzlKTURyVnU5ZVNmcEYuVno2TWtvTDRDZ0xjOXdjVHFBZWVKdUNDblpSeTIyWnU5OFA0bm1xRnFlNGI1WFgyQS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTd

### Get mediator grant response

In [13]:
mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
mediator_did = mediator_unpack_msg.message.from_prior.sub

https://didcomm.org/coordinate-mediation/2.0/mediate-grant
Routing Key: did:peer:2.Ez6LSgHf3aYDeUuPMqrjaQdkGJgte3q1gP1JDDCPu1XzTzuKJ.Vz6MkrUWx91o6GkCztCzukk1zpUmi9WsACgWaZ8JXUcWmCdko.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0
Mediator new DID: did:peer:2.Ez6LSt6cEb5NxaYSdGxFHEM7NGt6GNs1LMu79JMDrVu9eSfpF.Vz6MkoL4CgLc9wcTqAeeJuCCnZRy22Zu98P4nmqFqe4b5XX2A.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0


### Keylist update

Alice create a new DID to share with  other peers using mediator routing keys 

In [14]:
alice_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
# alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=mediator_routing_key)

print("Alice's DID for Bob:", alice_did_new)

Alice's DID for Bob: did:peer:2.Ez6LSbhLuMKZzK5veas6VYsX8S97TxrMND3Rk817uUg3P1tnn.Vz6MkeuPCzyD5PFG3Qc6QnNoVLo3HxvpTadsDCRmfeGhBhnTr.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2dIZjNhWURlVXVQTXFyamFRZGtHSmd0ZTNxMWdQMUpERENQdTFYelR6dUtKLlZ6Nk1rclVXeDkxbzZHa0N6dEN6dWtrMXpwVW1pOVdzQUNnV2FaOEpYVWNXbUNka28uU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19


Alice create and send the keylist update message

In [15]:
alice_keylist_update = Message(
    id = "unique-id-293e9a922efff",
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": alice_did_new,
            "action": "add"
        }
    ]
    }
)

In [16]:
alice_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_keylist_update,
    frm = alice_did_for_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [17]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_keylist_update_packed_msg.packed_msg)

In [18]:
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)


https://didcomm.org/coordinate-mediation/2.0/keylist-update-response
{'updated': [{'recipient_did': 'did:peer:2.Ez6LSbhLuMKZzK5veas6VYsX8S97TxrMND3Rk817uUg3P1tnn.Vz6MkeuPCzyD5PFG3Qc6QnNoVLo3HxvpTadsDCRmfeGhBhnTr.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2dIZjNhWURlVXVQTXFyamFRZGtHSmd0ZTNxMWdQMUpERENQdTFYelR6dUtKLlZ6Nk1rclVXeDkxbzZHa0N6dEN6dWtrMXpwVW1pOVdzQUNnV2FaOEpYVWNXbUNka28uU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19', 'action': 'add', 'result': 'success'}]}


### Alice can now send her DID to Bob. Copy Alice's DID into Bob's notebook, and follows Bob's notebook

In [20]:
#get nessus
nessus_invitation = json.loads(requests.get('http://nessus-tech.io:9100/message/invitation?inviter=Government&method=peer').text)
nessus_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))

In [21]:
nessus_did_doc

{'id': 'did:peer:2.Ez6LSh3pStyQygF6KTz8R4V8tQeTCQpV6sy7XBFiH5cJkMMBg.Vz6MkpVEyR4LPQJFh3DnkpScaie83qFYo6vuUYwwUpLjFNhUN.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ',
 'authentication': [{'id': 'did:peer:2.Ez6LSh3pStyQygF6KTz8R4V8tQeTCQpV6sy7XBFiH5cJkMMBg.Vz6MkpVEyR4LPQJFh3DnkpScaie83qFYo6vuUYwwUpLjFNhUN.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ#6MkpVEyR4LPQJFh3DnkpScaie83qFYo6vuUYwwUpLjFNhUN',
   'type': 'Ed25519VerificationKey2020',
   'controller': 'did:peer:2.Ez6LSh3pStyQygF6KTz8R4V8tQeTCQpV6sy7XBFiH5cJkMMBg.Vz6MkpVEyR4LPQJFh3DnkpScaie83qFYo6vuUYwwUpLjFNhUN.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ',
   'publicKeyMultibase': 'z6MkpVEyR4LPQJFh3DnkpScaie83qFYo6vuUYwwUpLjFNhUN'}],
 'keyAgreement': [{'id': 'did:peer:2.Ez6LSh3pStyQygF6KTz8R4V8tQeTCQpV6sy7XBFiH5cJkMMBg.Vz6MkpVEyR4LPQJFh3DnkpScaie83qFYo6vuUYwwUpLjFNhUN.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ#6LSh3pStyQygF6KTz8R4V8tQeTCQpV6sy7XBFiH5cJkMMBg',
   'type': '

In [22]:
ping_msg = Message(
    body = { "response_requested": True },
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/trust-ping/2.0/ping",
    frm = alice_did_new,
    to = [nessus_invitation["from"]],
)

tony_trust_ping_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = ping_msg,
    frm = alice_did_new,
    to = nessus_invitation['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_trust_ping_packed.packed_msg))
resp.text

'{}'

### Alice check status of queued messages from the Mediator

In [23]:
alice_status_check = Message(
    id = "unique-id-293e9a922efffxxx",
    type = "https://didcomm.org/messagepickup/3.0/status-request",
    body = {}
)

In [24]:
alice_status_check_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_status_check,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp3 = requests.post(mediator_endpoint, headers=headers, data = alice_status_check_packed_msg.packed_msg)

In [25]:
mediator_unpack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp3.json()
)
print(mediator_unpack_status.message.type)
print("Messages in Mediator queue: " + str(mediator_unpack_status.message.body["message_count"]))

https://didcomm.org/messagepickup/3.0/status
Messages in Mediator queue: 1


### If there are messages, Alice can pickup from the mediator (delivery request)

In [26]:
alice_delivery_request = Message(
    id = "unique-id-293e9a922efffxxxff",
    type = "https://didcomm.org/messagepickup/3.0/delivery-request",
    body = {"limit": 1}
)

In [27]:
alice_delivery_request_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_delivery_request,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp4 = requests.post(mediator_endpoint, headers=headers, data = alice_delivery_request_packed_msg.packed_msg)

In [28]:
mediator_delivery = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp4.json()
)
print(mediator_delivery.message.type)
print(mediator_delivery.message.body)
bob_packed_msg = mediator_delivery.message.attachments[0].data.json
msg_id = mediator_delivery.message.attachments[0].id

https://didcomm.org/messagepickup/3.0/delivery
{'recipient_key': None}


In [29]:
bob_packed_msg = mediator_delivery.message.attachments[0].data.json
msg_id = mediator_delivery.message.attachments[0].id

In [30]:
print(bob_packed_msg)

{'ciphertext': '3t3r_aLgjqmNZ8HXiOPqtLA4MM9kUoiLsI2Y9C-iQnTIvpx_7WWqd-y8-6MW73jw7ZV3Tk6KRYs11zycTZFy6-yrPTmluv07YCfSXHiVacF6xoSF2x4KhLQKbqY3Fv6ULyOqmEjqsiqH8X2KpA4lCmkihxAs4pOVn5SW3457V5leaSn45ssh8Bq70wYXwLgh8MfL-CMsajf11ClXtbocZHkQOYKV9_yLsd_PEiwHqAtKt_uiSCa8TfQOiffEHgzlWhtIqrLa2-Uj6sI5wKOvsk4JUfn2kviP_o3jzpw1B4c0gyyUcon66mqk8c6ONAR8nEmPS8Eas6Q1y4ZB9e8TgljmqNgTqilpY2zXKzmIfap_XewRuf6O4TjOKZa3jcHoLubQ0Nqyw64c7Jm14MeRK88hYMGYFB4UGxwfdpLhAyecXX3pAiPGO269N5k9wUWf-Tu9x3WNjaQLLBmty2NBpqzu20GMAcGTw1plEwWi_zmCz4NaTvBe08mwzxo03nTAfo7s1XcxhPIz0_1gRMfX1huQYmiwYI_oU0nbfxIec8cLwenhoDWV6QSv4Hv40yU1QL12iAg4JE-rR1-btDOUQSL2uWrOmPPRO6qZno_n4tChFHH8jX7wCCMGM5pS8716fDREnOidb5BWONfOseglGC_WN-oNzp75EhNOvSwDDblqJYdY9hjDNo5PMk3jir91TD2T6rJ_YKPeNTjVQIKh0ddvXTYh1-jq_UIe9ywuJ_-K2XdZqr05jHTfyAfwJ_lDGipPPLmL2HLnUoEkmJjusqkbNUrd3pSHvbgM06iglfoG9SDOzDEazfURHkAi6OV5noExIGz3PT4cTEyG5Z3RuKfrTQ44ZVhV9yqm68ljrLXUm7gWSOGWe8npmIT4kHLAFhulKOqA3xHVNpSWkQdqKUXRLtfQOiqfVuuQWXjx1Mv2k6hMizAfmDNOP5wbfozS6H5LBg1TQStE7TWOKE3gbunT

In [31]:
bob_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= bob_packed_msg
)

In [32]:
print("Message ID:", msg_id)

Message ID: 642de10afb5566e28c19e1a6


In [34]:
bob_msg.message.as_dict()['body']

{'comment': 'Pong from Government'}

### Alice acknowledge so the mediator can delete the message

In [35]:
alice_ack = Message(
    id = "unique-id-293e9a922efffxxxffsss",
    type = "https://didcomm.org/messagepickup/3.0/messages-received",
    body = {"message_id_list": [msg_id]}
)

In [36]:
alice_ack_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_ack,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp5 = requests.post(mediator_endpoint, headers=headers, data = alice_ack_packed_msg.packed_msg)

In [37]:
mediator_ack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp5.json()
)
print(mediator_ack_status.message.type)
print(mediator_ack_status.message.body)

https://didcomm.org/messagepickup/3.0/status
{'message_count': 0, 'live_delivery': False}


## Send basic message to Nessus

In [38]:

tony_basic_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/basicmessage/2.0/message",
    body={"content": f'Hey Nessus, this is RootsID {str(uuid.uuid4())[:5]}'},
    created_time= int(datetime.datetime.now().timestamp()),
    to = [nessus_invitation['from']],
        frm = alice_did_new,
)

tony_basic_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = tony_basic_message,
    frm = alice_did_new,
    to = nessus_invitation['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_basic_message_packed.packed_msg))
resp.text

'{}'